In [12]:
import torchvision

In [13]:
torchvision.__version__

'0.21.0+cu124'

In [21]:
!which python

/home/unsw/anaconda3/envs/qa/bin/python


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [22]:
pwd

'/media/unsw/17D1-7AB7/mostafa/audioLLM/Qwen'

In [1]:
!nvidia-smi

Wed Mar  5 10:14:35 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.216.01             Driver Version: 535.216.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               On  | 00000000:04:00.0 Off |                  Off |
| 30%   45C    P8              15W / 300W |   7356MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Trelis Fine-tune Bird dataset

In [ ]:
!pip install peft hf_transfer

In [ ]:
!pip install datasets

In [28]:
!pip install tensorboardX

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [1]:
# Enable fast download
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import torch, gc
from PIL import Image
from transformers import Qwen2AudioForConditionalGeneration, AutoProcessor, BitsAndBytesConfig

device = 'cuda'

torch_dtype = torch.bfloat16 #Brain Floating Point: bfloat16 is a 16-bit floating-point format designed for machine learning. It's like a shorter version of the standard 32-bit floating-point (float32) format.

model_id = 'Qwen/Qwen2-Audio-7B-Instruct'

model = Qwen2AudioForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype = torch_dtype,
    trust_remote_code = True
    
).to(device)

processor = AutoProcessor.from_pretrained(model_id, trust_remote_code = True)

/home/unsw/anaconda3/envs/qa/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  4.49it/s]


In [3]:
from IPython.display import Audio, display
from urllib.request import urlopen
from io import BytesIO
import librosa

In [ ]:
TASK_PROMPT = "Identify the bird species in the audio clip"
debug = False

def run_example(audio_url):
    try:
        #load audio
        audio_data, sample_rate = librosa.load(BytesIO(urlopen(audio_url).read()), sr=None)
        #play audio
        display(Audio(audio_data, rate=sample_rate))
        conversation = [
            {"role": "user", "content": [
                {"type": "audio", "audio_url": audio_url},
                {"type": "text", "text": TASK_PROMPT},
            ]}
        ]
        # process conversation
        text = processor.apply_chat_template(conversation, add_generation_prompt= True, tokenize=False)
        if debug:
            print(f"Templated text:\n{text}")
            
        audios = []
    
        for message in conversation:
            if isinstance(message["content"], list):
                for item in message["content"]:
                    if isinstance(item, dict) and item["type"] == "audio" and "audio_url" in item:
                        try:
                            sr = int(processor.feature_extractor.sampling_rate)
                            aduio_data,_ = librosa.load(BytesIO(urlopen(item["audio_url"]).read()), sr=sr)
                            audios.append(aduio_data)
                        except Exception as e:
                            print(f"failed to process audio from {item['audio_url']}, {e}")
        inputs = processor(
            text=text,
            audios=audios,
            return_tensors="pt",
            padding=True,
        ).to("cuda")

        generate_ids = model.generate(**inputs, max_length=1024)

        print(f"Input shape: {inputs.input_ids.size(1)}, Generated shape: {generate_ids.shape}")
        generate_ids = generate_ids[:, inputs.input_ids.size(1):]
        
        response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
        print(response)
    except Exception as e:
        print(f"An error occurred: {e}")
       
audio_url = "https://xeno-canto.org/841748/download"
run_example(audio_url)
                        

In [5]:
from datasets import load_dataset

ds = load_dataset("Trelis/bird-songs")

train_dataset = ds["train"]
eval_dataset = ds["validation"]

In [6]:
import torch
from PIL import Image
from torchvision.transforms.functional import to_pil_image, resize

def run_model_evaluation(dataset):
    for example in dataset:
        audio = example["url"]
        results = run_example(audio)
        print(f"Ground truth answer: {example['name']}")
        print("\n")
    return

In [ ]:
run_model_evaluation(eval_dataset)

In [7]:
print(processor.tokenizer.bos_token)
print(processor.tokenizer.bos_token_id)
print("___")

print(processor.tokenizer.eos_token)
print(processor.tokenizer.eos_token_id)
print("___")

print(processor.tokenizer.pad_token)
print(processor.tokenizer.pad_token_id)

None
None
___
<|im_end|>
151645
___
<|endoftext|>
151643


In [8]:
print(processor.feature_extractor.n_samples)

480000


In [9]:
print(processor.tokenizer.chat_template)

{% for message in messages %}{% if loop.first and messages[0]['role'] != 'system' %}{{ '<|im_start|>system
You are a helpful assistant.<|im_end|>
' }}{% endif %}{{'<|im_start|>' + message['role'] + '
' + message['content'] + '<|im_end|>' + '
'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant
' }}{% endif %}


In [10]:
import numpy as np
from typing import Any, Dict, List
from torch.utils.data import DataLoader
debug = False
class AudioDataCollator:
    def __init__(self, processor, task_prompt: str, max_length: int = None):
        self.processor = processor
        self.task_prompt = task_prompt
        self.max_length = max_length or processor.feature_extractor.n_samples
        self.sampling_rate = processor.feature_extractor.sampling_rate

    def process_audio(self, audio_url: str) -> np.ndarray:
        try:
            audio, sr = librosa.load(BytesIO(urlopen(audio_url).read()), sr=self.sampling_rate)
            
            if len(audio) > self.max_length:
                if debug:
                    print(f"Trimming audio to {self.max_length} samples")
                audio = audio[:self.max_length]
            return audio.astype(np.float32)
            
        except Exception as e:
            raise RuntimeError(f"Error processing audio: {e}")

    def __call__(self, examples: List[Dict[str,Any]]) -> Dict[str, torch.Tensor]:
        valid_examples = []
        audios = []
        combined_texts = []
        for example in examples:
            try:
                audio = self.process_audio(example["url"])
                audios.append(audio)

                conversation = [
                    {"role": "user", "content":[
                        {"type": "audio", "audio_url": example["url"]},
                        {"type": "text", "text": self.task_prompt}
                    ]},
                    {"role": "assistant", "content": example["name"]}
                ]
                combined_text = self.processor.apply_chat_template(
                    conversation, add_generation_prompt=False, tokenize=False  #Here add_generation_prompt=False as it's training and the answer is in the conversation, and tokenize=false as we will do tokenize later
                )

                combined_texts.append(combined_text)
                valid_examples.append(example)
            except Exception as e:
                print(f"Error processing example {example['id']}: {e}")
                continue
        if not valid_examples:
            raise ValueError("No valid examples found")

        if debug:
            print(f"Number of valid examples: {len(valid_examples)}")
            print(f"Number of audios: {len(audios)}")
            print(f"Number of combined texts: {len(combined_texts)}")

        try:
            inputs = self.processor(
                text=combined_texts,
                audios=audios,
                return_tensors="pt",
                padding=True
            )
        except Exception as e:
            raise RuntimeError(f"Error processing inputs: {e}")

        labels = inputs["input_ids"].clone()

        if debug:
            print(f"Input ids shape: {inputs['input_ids'].shape}")
            print(f"Attention mask shape: {inputs['attention_mask'].shape}")
            print(f"Labels shape: {labels.shape}")

        # Mask the prompt portion
        for i in range(len(combined_texts)):
            try:
                input_ids_row = inputs["input_ids"][i]
                if debug:
                    print(f"\nProcessing example {i}")
                    print(f"Input IDs for this example: {input_ids_row}")

                assistant_start_tokens = self.processor.tokenizer.encode("<|im_start|>assistant")
                if debug:
                    print(f"Assistant start tokens: {assistant_start_tokens}")

                assistant_start_idx = -1
                for j in range(len(input_ids_row) - len(assistant_start_tokens) + 1):
                    if torch.equal(input_ids_row[j:j+len(assistant_start_tokens)], torch.tensor(assistant_start_tokens)):
                        assistant_start_idx = j
                        break
                if debug:
                    print(f"Assistant start index: {assistant_start_idx}")

                if assistant_start_idx != -1:
                    labels[i, :assistant_start_idx + len(assistant_start_tokens)] = -100
                else:
                    print(f"Error: Assistant start index not found for example {i}")
                    labels[i,:] = -100
            except Exception as e:
                print(f"Error processing example {i}: {e}")

        if debug:
            print(f"Input ids shape after masking: {inputs['input_ids'].shape}")
            print(f"Attention mask shape after masking: {inputs['attention_mask'].shape}")
            print(f"Labels shape after masking: {labels.shape}")
            print(f"First label row after masking: {labels[0]}")

        return {
            "input_ids": inputs["input_ids"],
            "attention_mask": inputs["attention_mask"],
            "input_features": inputs["input_features"],
            "feature_attention_mask": inputs["feature_attention_mask"],
            "labels": labels
        }

                
          
    

In [11]:
def inspect_batch(batch: Dict[str, torch.Tensor], processor) -> None:
    print("\n===== INSPECTING BATCH ====")
    
    for key, tensor in batch.items():
        print(f"\n{key}")
        print(f" shape: {tensor.shape}")
        print(f" dtype: {tensor.dtype}")
        print(f" device: {tensor.device}")
        if key not in ['input_features', 'feature_attention_mask']:
            print(f" First element: {tensor[0].tolist()}")

    if "input_ids" in batch:
        print("\n=== Decoded input_ids ===")
        first_input = batch["input_ids"][0].tolist()
        decoded_input = processor.tokenizer.decode(first_input, skip_special_tokens=False)
        print(f"Decoded Input (First Row):\n{decoded_input}")

    if "labels" in batch:
        print("\n=== Decoded labels ===")
        first_label = batch["labels"][0].tolist()
        valid_tokens = [token for token in first_label if token != -100]
        decoded_label = processor.tokenizer.decode(valid_tokens, skip_special_tokens=False)
        print(f"Decoded Label (First Row):\n{decoded_label}")

    

In [12]:
data_collator = AudioDataCollator(
    processor=processor,
   task_prompt = "Identify the bird species in this audio clip:"    
)

In [13]:
dataloader = DataLoader(
    train_dataset,
    batch_size=1,
    collate_fn=data_collator,
    shuffle=True
)

In [ ]:
batch = next(iter(dataloader))

In [ ]:
inspect_batch(batch, processor)

In [42]:
processor.tokenizer.encode("<|im_start|>assistant", add_special_tokens=False)

[151644, 77091]

In [14]:
print(model)

Qwen2AudioForConditionalGeneration(
  (audio_tower): Qwen2AudioEncoder(
    (conv1): Conv1d(128, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
    (embed_positions): Embedding(1500, 1280)
    (layers): ModuleList(
      (0-31): 32 x Qwen2AudioEncoderLayer(
        (self_attn): Qwen2AudioSdpaAttention(
          (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
          (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
          (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
          (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (activation_fn): GELUActivation()
        (fc1): Linear(in_features=1280, out_features=5120, bias=True)
        (fc2): Linear(in_features=5120, out_features=1280, bias=True)
        (final_layer_norm): Layer

In [15]:
import peft

In [16]:
lora_config = peft.LoraConfig(
    r=32,
    #target_module_size=64,
    lora_alpha=32,
    use_rslora=True,
    target_modules="all-linear",
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

In [17]:
lora_config

LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=32, target_modules='all-linear', exclude_modules=None, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=True, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, eva_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)

In [18]:
model = peft.get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 108,843,008 || all params: 8,505,937,920 || trainable%: 1.2796


In [22]:
from transformers import TrainingArguments, Trainer

lr = 2e-5

# epochs=1
# scheduler = "constant"

#optional

epochs =1
scheduler = "cosine"

batch_size = 4

run_name = f"run_lr_{lr}_epochs_{epochs}_scheduler_{scheduler}_batch_size_{batch_size}"

training_args = TrainingArguments(
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=lr,
    weight_decay=0.01,
    logging_steps=1,
    output_dir=f"./results/{run_name}",
    eval_strategy="steps",
    eval_steps=0.2,
    lr_scheduler_type=scheduler,
    bf16=True,
    remove_unused_columns=False,
    report_to="tensorboard",
    run_name=run_name,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False}
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

In [23]:
trainer.train()

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Step,Training Loss,Validation Loss
4,0.191600,0.537067
8,0.375400,0.763963
12,0.404700,0.741654
16,0.988700,0.655982
20,0.521000,0.632780


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the `sampling_rate` argument 

TrainOutput(global_step=20, training_loss=0.38316794894635675, metrics={'train_runtime': 475.7483, 'train_samples_per_second': 0.168, 'train_steps_per_second': 0.042, 'total_flos': 2995776001646592.0, 'train_loss': 0.38316794894635675, 'epoch': 1.0})

In [ ]:
print(model)

In [ ]:
model.eval()
run_model_evaluation(eval_dataset)

# CI TestBench

## Create Dataset for testing

In [28]:
# Dataset should include these columns
# - Audio
# - Prompt
# - Answer
# - text
# - language
# - age
# - gender
# - Type of assessment


In [8]:
# TAUKADIAL data
# Create single GT file
test_gt = '/media/unsw/17D1-7AB7/mostafa/datasets/TAUKADIAL-24/test_groudtruth.csv'
test_md = '/media/unsw/17D1-7AB7/mostafa/datasets/TAUKADIAL-24/test_meta.csv'

test_gt_df = pd.read_csv(test_gt)
test_md_df = pd.read_csv(test_md)

test_df = pd.merge(test_gt_df, test_md_df, how='left', on='tkdname')
test_df.to_csv('/media/unsw/17D1-7AB7/mostafa/datasets/TAUKADIAL-24/test_groundtruth.csv', index=False)

In [6]:
print(test_df)

                tkdname  mmse   dx  age sex
0    taukdial-001-1.wav    27   NC   70   F
1    taukdial-001-2.wav    27   NC   70   F
2    taukdial-001-3.wav    27   NC   70   F
3    taukdial-010-1.wav    27  MCI   91   F
4    taukdial-010-2.wav    27  MCI   91   F
..                  ...   ...  ...  ...  ..
115  taukdial-166-2.wav    28  MCI   88   M
116  taukdial-166-3.wav    28  MCI   88   M
117  taukdial-168-1.wav    29   NC   65   M
118  taukdial-168-2.wav    29   NC   65   M
119  taukdial-168-3.wav    29   NC   65   M

[120 rows x 5 columns]


In [24]:
# TAUKADIAL data
import pandas as pd
train_gt = '/media/unsw/17D1-7AB7/mostafa/datasets/TAUKADIAL-24/train_groundtruth.csv'
train_path = '/media/unsw/17D1-7AB7/mostafa/datasets/TAUKADIAL-24/train'

test_gt = '/media/unsw/17D1-7AB7/mostafa/datasets/TAUKADIAL-24/test_groundtruth.csv'
test_path = '/media/unsw/17D1-7AB7/mostafa/datasets/TAUKADIAL-24/test'

In [36]:
import pandas as pd
from datasets import Dataset, Audio
import librosa
import numpy as np

def create_audio_dataset(csv_file, audio_dir, target_sr=16000):
    """
    Creates a Hugging Face Dataset from a CSV file and audio files,
    converting to mono and resampling to the target sampling rate using Librosa.

    Args:
        csv_file (str): Path to the CSV file.
        audio_dir (str): Path to the directory containing audio files.
        target_sr (int): Target sampling rate for resampling (default: 16000).

    Returns:
        datasets.Dataset: Hugging Face Dataset.
    """

    df = pd.read_csv(csv_file)

    data = {
        "audio": [],
        "age": [],
        "sex": [],
        "mmse": [],
        "dx": [],
        "tkdname": []
    }

    for index, row in df.iterrows():
        audio_path = f"{audio_dir}/{row['tkdname']}"
        try:
            # Load audio using librosa, converting to mono and resampling
            audio_array, _ = librosa.load(audio_path, sr=target_sr, mono=True)

            data["audio"].append({"path": audio_path, "array": audio_array, "sampling_rate": target_sr})
            data["age"].append(row["age"])
            data["sex"].append(row["sex"])
            data["mmse"].append(row["mmse"])
            data["dx"].append(row["dx"])
            data["tkdname"].append(row['tkdname'])
        except FileNotFoundError:
            print(f"Warning: Audio file not found: {audio_path}")
        except Exception as e: # Catch other librosa related errors.
            print(f"Warning: Error processing {audio_path}: {e}")

    dataset = Dataset.from_dict(data)
    dataset = dataset.cast_column("audio", Audio(sampling_rate=target_sr)) #cast the audio column to audio type.

    return dataset

In [37]:
# create train dataset

train_audio_dataset = create_audio_dataset(train_gt, train_path, target_sr=16_000)

print(train_audio_dataset)
print(train_audio_dataset[0]) #prints the first element of the dataset.

#optional save to disk.
#audio_dataset.save_to_disk("your_output_dataset_directory")

Dataset({
    features: ['audio', 'age', 'sex', 'mmse', 'dx', 'tkdname'],
    num_rows: 387
})
{'audio': {'path': None, 'array': array([0.00143433, 0.00158691, 0.00149536, ..., 0.0206604 , 0.02038574,
       0.01937866]), 'sampling_rate': 16000}, 'age': 72, 'sex': 'F', 'mmse': 29, 'dx': 'NC', 'tkdname': 'taukdial-002-1.wav'}


In [25]:
# create test dataset

test_audio_dataset = create_audio_dataset(test_gt, test_path, target_sr=16_000)

print(test_audio_dataset)
print(test_audio_dataset[0]) #prints the first element of the dataset.

#optional save to disk.
#audio_dataset.save_to_disk("your_output_dataset_directory")

Dataset({
    features: ['audio', 'age', 'sex', 'mmse', 'dx', 'tkdname'],
    num_rows: 120
})
{'audio': {'path': None, 'array': array([-0.00991821, -0.01715088, -0.01513672, ..., -0.00558472,
       -0.00704956,  0.        ]), 'sampling_rate': 16000}, 'age': 70, 'sex': 'F', 'mmse': 27, 'dx': 'NC', 'tkdname': 'taukdial-001-1.wav'}


In [38]:
from datasets import DatasetDict
taukadial_ds = DatasetDict({'train':train_audio_dataset, 'test':test_audio_dataset})

In [ ]:
from IPython import display
display.Audio(data=train_audio_dataset[0]['audio']['array'], rate=16_000)

In [40]:
taukadial_ds.save_to_disk("/media/unsw/17D1-7AB7/mostafa/datasets/TAUKADIAL_dataset")

Saving the dataset (1/1 shards): 100%|████████████████████████████████████████████████████████████████████████████████████████████| 120/120 [00:02<00:00, 54.96 examples/s]


# Create dataset from chuncks

In [27]:
train_path = '/media/unsw/17D1-7AB7/mostafa/datasets/TAUKADIAL-24/train'
train_chuncks_cvs = "/media/unsw/17D1-7AB7/mostafa/datasets/TAUKADIAL-24/train_chunks.csv"
test_path = '/media/unsw/17D1-7AB7/mostafa/datasets/TAUKADIAL-24/test'
test_chuncks_cvs = "/media/unsw/17D1-7AB7/mostafa/datasets/TAUKADIAL-24/test_chunks.csv"

In [22]:
from datasets import DatasetDict, Dataset, Audio
import pandas as pd
import torchaudio
import torchaudio.transforms as T
import os
import numpy as np

def load_and_trim_audio(file_path, start_sample, end_sample, target_sample_rate=16000):
    """
    Loads an audio file, converts it to mono, resamples to 16kHz, and trims to the specified sample range.

    Args:
        file_path (str): Path to the audio file.
        start_sample (int): Start sample index.
        end_sample (int): End sample index.
        target_sample_rate (int, optional): Desired sampling rate (default: 16kHz).

    Returns:
        dict: A dictionary containing 'array' (trimmed waveform) and 'sampling_rate' (16kHz).
    """
    # Load audio file
    waveform, sample_rate = torchaudio.load(file_path)

    # Convert to mono by averaging channels if it's stereo
    if waveform.shape[0] > 1:  # More than 1 channel means stereo
        waveform = waveform.mean(dim=0, keepdim=True)  # Convert to mono

    # Resample if needed
    if sample_rate != target_sample_rate:
        resampler = T.Resample(orig_freq=sample_rate, new_freq=target_sample_rate)
        waveform = resampler(waveform)

    # Ensure start and end samples are within bounds
    total_samples = waveform.shape[1]
    start_sample = min(start_sample, total_samples - 1)
    end_sample = min(end_sample, total_samples)

    # Trim the waveform
    trimmed_waveform = waveform[:, start_sample:end_sample]

    return {"array": trimmed_waveform.numpy().squeeze(), "sampling_rate": target_sample_rate}

def create_hf_audio_dataset(csv_path, audio_base_path=None):
    """
    Creates a Hugging Face dataset from a CSV file containing segmented audio metadata.

    Args:
        csv_path (str): Path to the CSV file.
        audio_base_path (str, optional): Base directory for audio files (if paths need to be adjusted).

    Returns:
        DatasetDict: A Hugging Face Dataset with processed audio segments.
    """

    # Load CSV into a Pandas DataFrame
    df = pd.read_csv(csv_path)

    # Adjust file paths if needed
    if audio_base_path:
        df["file_path"] = df["file_path"].apply(lambda x: os.path.join(audio_base_path, os.path.basename(x)))

    # Process audio: convert to mono, resample, and trim
    df["audio"] = df.apply(
        lambda row: load_and_trim_audio(row["file_path"], row["start_sample"], row["end_sample"]),
        axis=1
    )

    # Rename text column for consistency
    df = df.rename(columns={"text": "transcription"})

    # Convert DataFrame to Hugging Face Dataset
    dataset = Dataset.from_pandas(df)

    return dataset




In [28]:
#train_dataset = create_hf_audio_dataset(train_chuncks_cvs,train_path)
test_dataset = create_hf_audio_dataset(test_chuncks_cvs,test_path)


In [29]:
audio_dataset = DatasetDict({'train':train_dataset, 'test':test_dataset})

In [ ]:
from IPython.display import Audio
Audio(data=audio_dataset['test'][0]['audio']['array'], rate=16_000)

In [33]:
audio_dataset['test'][0]['transcription']

" to the lovely day. The mother has fallen asleep holding one of the children. And in the second window, the little girl is still blowing bubble gum. She was blowing bubble gum going to this wonderful day. And the bubble was much bigger than it is now. The dog that was going with them was so excited about being out. He's hanging out the window."

In [34]:
audio_dataset.save_to_disk('/media/unsw/17D1-7AB7/mostafa/datasets/TAUKADIAL_dataset_chuncks')

Saving the dataset (2/2 shards): 100%|████████████████████████████████████████████████████████████████████████████████████████████| 360/360 [00:00<00:00, 982.60 examples/s]


In [35]:
audio_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'file_path', 'file_id', 'spk_id', 'prompt_id', 'start_sample', 'end_sample', 'lang', 'transcription', 'tkdname', 'age', 'sex', 'mmse', 'dx', 'audio'],
        num_rows: 1032
    })
    test: Dataset({
        features: ['Unnamed: 0', 'file_path', 'file_id', 'spk_id', 'prompt_id', 'start_sample', 'end_sample', 'lang', 'transcription', 'tkdname_x', 'mmse', 'dx', 'tkdname_y', 'age', 'sex', 'audio'],
        num_rows: 360
    })
})

## Language Detection

In [1]:
from transformers import Qwen2AudioForConditionalGeneration, AutoProcessor
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct")
model = Qwen2AudioForConditionalGeneration.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct", device_map="auto")
model.tie_weights()
#model.to("cuda")

/home/unsw/anaconda3/envs/qa/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:09<00:00,  1.99s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


In [2]:
model.device

device(type='cuda', index=0)

In [13]:
def detect_lang_of_audio(audio, model, processor):
    conversation = [
        {'role': 'system', 'content': 'You are a helpful assistant.'}, 
        {"role": "user", "content": [
            {"type": "audio", "audio": "audio.wav"},
            {"type": "text", "text": "What is the language spoken in the audio? choose from English or Mandarine. Give answer as a single word"},
        ]},
    ]
    text = processor.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False)
    audios = [audio]
    inputs = processor(text=text, audios=audios, return_tensors="pt", padding=True)
    inputs.input_ids = inputs.input_ids.to(model.device)
    generate_ids = model.generate(**inputs, max_length=1024)
    generate_ids = generate_ids[:, inputs.input_ids.size(1):]
    response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return response

In [16]:
def detect_lang_batch(batch, model, processor):
    audio = batch['audio']["array"]
    batch['lang'] = detect_lang_of_audio(audio, model, processor)
    return batch

In [ ]:
taukadial_data = taukadial_data.map(detect_lang_batch, fn_kwargs={'model':model, 'processor':processor})

In [26]:
taukadial_data.save_to_disk("/media/unsw/17D1-7AB7/mostafa/datasets/TAUKADIAL_dataset_lang")

Saving the dataset (1/1 shards): 100%|███████████████████████████████████████████████████████████████████████████████████████████| 120/120 [00:00<00:00, 177.09 examples/s]


In [14]:
# Load Dataset
from datasets import load_from_disk, Dataset
taukadial_data = load_from_disk("/media/unsw/17D1-7AB7/mostafa/datasets/TAUKADIAL_dataset")

In [15]:
audio = taukadial_data['train'][3]["audio"]["array"]
print(detect_lang_of_audio(audio, model, processor))

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Mandarin


In [ ]:
from IPython import display
display.Audio(data=audio, rate=16_000)

In [9]:
data_part = Dataset.from_dict(taukadial_data['train'][:10])

In [ ]:
data_part = data_part.map(detect_lang_batch, fn_kwargs={'model':model, 'processor':processor})

## Zero-shot CI detection

In [27]:
conversation = [
    {'role': 'system', 'content': 'You are a helpful assistant.'}, 
    {"role": "user", "content": [
        {"type": "audio", "audio": "audio.wav"},
        {"type": "text", "text": "Based on the input audio, indicate only the diagnosis. Choose from: Normal Cognitive Decline or Mild Cognitive Impairment. Give the \
        answer as single word. Use NC for Normal Cognitive Decline and MCI for Mild Cognitive Impairment"},
    ]},
]

In [37]:
def detect_CI_of_audio(audio, prompt, model, processor):
    conversation = [
        {'role': 'system', 'content': 'You are a helpful assistant.'}, 
        {"role": "user", "content": [
            {"type": "audio", "audio": "audio.wav"},
            {"type": "text", "text": prompt},
        ]},
    ]
    text = processor.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False)
    audios = [audio]
    inputs = processor(text=text, audios=audios, return_tensors="pt", padding=True)
    inputs.input_ids = inputs.input_ids.to(model.device)
    generate_ids = model.generate(**inputs, max_length=1024)
    generate_ids = generate_ids[:, inputs.input_ids.size(1):]
    response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return response

In [39]:
# Load Dataset
from datasets import load_from_disk
taukadial_data = load_from_disk("/media/unsw/17D1-7AB7/mostafa/datasets/TAUKADIAL_dataset_lang")

In [45]:
PROMPT = "Based on the input audio, indicate only the diagnosis. \
Choose from: Normal Cognitive Decline or Mild Cognitive Impairment. \
Give the answer as a single word. Use NC for Normal Cognitive Decline and MCI for Mild Cognitive Impairment"
def detect_CI_batch(batch, prompt, model, processor):
    audio = batch['audio']["array"]
    batch['dx_detect'] = detect_CI_of_audio(audio, prompt, model, processor)
    return batch

In [40]:
data_part = Dataset.from_dict(taukadial_data['train'][:10])

In [41]:
data_part = data_part.map(detect_CI_batch, fn_kwargs={'prompt': PROMPT,'model':model, 'processor':processor})

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.?, ? examples/s]
/home/unsw/anaconda3/envs/qa/lib/python3.11/site-packages/transformers/generation/utils.py:2110: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.2.79s/ examples]
It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to de

In [44]:
data_part['dx']

['NC', 'NC', 'NC', 'MCI', 'MCI', 'MCI', 'NC', 'NC', 'NC', 'NC']

In [ ]:
taukadial_data_test_detect = taukadial_data['test'].map(detect_CI_batch, fn_kwargs={'prompt': PROMPT,'model':model, 'processor':processor})

In [ ]:
taukadial_data_test_detect['dx_detect']

In [33]:
text = processor.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False)
audios = [taukadial_data['train'][1]["audio"]["array"]]
inputs = processor(text=text, audios=audios, return_tensors="pt", padding=True)
inputs.input_ids = inputs.input_ids.to(model.device)


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [34]:
generate_ids = model.generate(**inputs, max_length=1024)
generate_ids = generate_ids[:, inputs.input_ids.size(1):]
response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

In [35]:
print(response)

MCI


In [36]:
taukadial_data['train'][1]

{'audio': {'path': None,
  'array': array([ 0.00540161,  0.00366211,  0.00131226, ..., -0.00674438,
         -0.00759888, -0.00765991]),
  'sampling_rate': 16000},
 'age': 72,
 'sex': 'F',
 'mmse': 29,
 'dx': 'NC',
 'tkdname': 'taukdial-002-2.wav'}

## Compute UAR

In [138]:
from sklearn.metrics import recall_score, f1_score, classification_report
from datasets import load_from_disk
for i in range(5):
    result_data_path = f'results/exp19/pred_dataset/data_prompt_{i}/'
    results_data = load_from_disk(result_data_path)
    y_true = results_data['dx']
    y_pred = results_data['dx_pred']
    uar = recall_score(y_true, y_pred, average='macro')
    f1_macro = f1_score(y_true, y_pred, average='macro')
    print(f'{i+1} uar:{uar}, f1:{f1_macro}')

1 uar:0.28393282366211303, f1:0.29070179745247254
2 uar:0.24359777749926892, f1:0.2464679022293849
3 uar:0.3286752725905502, f1:0.3341603515108553
4 uar:0.27106571416635333, f1:0.29830489278163697
5 uar:0.2607260726072607, f1:0.2863269008741547


/home/unsw/anaconda3/envs/qa/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/unsw/anaconda3/envs/qa/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/unsw/anaconda3/envs/qa/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/unsw/anaconda

In [141]:
classification_report(y_true, y_pred,output_dict=True)

/home/unsw/anaconda3/envs/qa/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/unsw/anaconda3/envs/qa/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/unsw/anaconda3/envs/qa/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'MCI': {'precision': 0.5757575757575758,
  'recall': 0.28217821782178215,
  'f1-score': 0.3787375415282392,
  'support': 202.0},
 'NC': {'precision': 0.4619883040935672,
  'recall': 0.5,
  'f1-score': 0.48024316109422494,
  'support': 158.0},
 'Unknown': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 0.0},
 'accuracy': 0.37777777777777777,
 'macro avg': {'precision': 0.34591529328371434,
  'recall': 0.2607260726072607,
  'f1-score': 0.2863269008741547,
  'support': 360.0},
 'weighted avg': {'precision': 0.5258255065272609,
  'recall': 0.37777777777777777,
  'f1-score': 0.423287230115533,
  'support': 360.0}}

In [109]:
from sklearn.metrics import recall_score, f1_score
from datasets import load_from_disk
import numpy as np
for i in range(5):
    result_data_path = f'results/exp4/pred_dataset/data_prompt_{i}/'
    results_data = load_from_disk(result_data_path)
    y_true = results_data['dx']
    y_pred = results_data['dx_pred']
    uar = recall_score(y_true, y_pred, average='macro')
    f1_macro = f1_score(y_true, y_pred, average='macro')
    print(f'{i+1} uar:{uar}, f1:{f1_macro}')
    print(np.unique(y_pred))

1 uar:0.2701197438039543, f1:0.27883167720566093
['MCI' 'NC' 'Unknown']
2 uar:0.5154553049289892, f1:0.4529469228264409
['MCI' 'NC']
3 uar:0.5, f1:0.3442622950819672
['MCI']
4 uar:0.2598162071846282, f1:0.2309248023533738
['MCI' 'NC' 'Unknown']
5 uar:0.5183792815371763, f1:0.4928003438641736
['MCI' 'NC']


/home/unsw/anaconda3/envs/qa/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/unsw/anaconda3/envs/qa/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [87]:
np.unique(y_pred)

array(['MCI', 'NC'], dtype='<U3')

In [112]:
result_data_path = f'results/exp4/pred_dataset/data_prompt_0/'
results_data = load_from_disk(result_data_path)

In [130]:
results_data['dx']

['NC',
 'NC',
 'NC',
 'MCI',
 'MCI',
 'MCI',
 'NC',
 'NC',
 'NC',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'NC',
 'NC',
 'NC',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'NC',
 'NC',
 'NC',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'NC',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'MCI',
 'NC',
 'NC',
 'NC']

In [ ]:
results_data['response']

## results_data

In [142]:
from datasets import load_from_disk


In [205]:
data = load_from_disk("results/exp1/pred_dataset/data_prompt_0/")

In [214]:
import numpy as np
mask = np.isin(data['response'], ['MCI', 'NC'])
np.mean(mask) * 100

np.float64(100.0)

In [204]:
i = 97
print(data[i]['response'])
print(data[i]['dx'])

{'Transcription': 'She has a tricycle near her and a ladder that’s not tall enough but she has a tricycle near her and she’s trying to catch the cat if the cat finally jumps or falls. There’s a dog who’s attempting to climb up to go vertically up a tree where the dad is sitting on a branch. There are firemen coming and bringing ladders and there’s a firetruck in the background. Thank you so this picture also tells a story.', 'Analysis': 'The speaker has some difficulty describing the image, showing signs of confusion and word-finding difficulties. They struggle to maintain grammatical coherence and have occasional pauses. Vocabulary is limited, with frequent use of simple words.', 'Diagnosis': 'MCI (Mild Cognitive Impairment)'}
MCI


In [165]:
import json

json_string = """
{"Transcription": "Okay, there was a family who were going on a trip, mother, father, and three kids. And they are driving along, and the kids, one little boy is hanging out the window with his dog. And the little girl is chewing bubble gum, working on a big, making a big bubble. And the other little boy is waving at people. And then they keep driving until it gets to be dark. And the mother is asleep in the front seat. And the one of the girls is asleep in the back seat.", "Analysis": "The speaker appears to be describing a scene from a family road trip. The description is vivid and includes specific details about each child’s activity, which helps paint a clear picture in the listener's mind. There are no errors in grammar, vocabulary, or pronunciation; the speech flows smoothly and is easy to understand. The only pause in the speech occurs when the speaker describes the mother falling asleep; otherwise, the speech is continuous and coherent.", "Diagnosis": "NC"}
"""

try:
    data = json.loads(json_string)
    #print(data)
    print(data['Transcription'])
    print(data['Analysis'])
    print(data['Diagnosis'])

except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")

Okay, there was a family who were going on a trip, mother, father, and three kids. And they are driving along, and the kids, one little boy is hanging out the window with his dog. And the little girl is chewing bubble gum, working on a big, making a big bubble. And the other little boy is waving at people. And then they keep driving until it gets to be dark. And the mother is asleep in the front seat. And the one of the girls is asleep in the back seat.
The speaker appears to be describing a scene from a family road trip. The description is vivid and includes specific details about each child’s activity, which helps paint a clear picture in the listener's mind. There are no errors in grammar, vocabulary, or pronunciation; the speech flows smoothly and is easy to understand. The only pause in the speech occurs when the speaker describes the mother falling asleep; otherwise, the speech is continuous and coherent.
NC


In [217]:
from datasets import load_from_disk
import os
import numpy as np

def get_per_one_word(exp_path):
    output = []
    for prompts_id in range(5):
        data = load_from_disk(os.path.join(exp_path,'pred_dataset',f'data_prompt_{prompts_id}'))
        mask = np.isin(data['response'], ['MCI', 'NC'])
        output.append((prompts_id,np.mean(mask) * 100))
    return output

In [277]:
#Compute UAC and MF1 per prompt, return avg, std, max, mvote, index of max P
from sklearn.metrics import recall_score, f1_score
import numpy as np
def compute_metrics(exp_path, lang=None):
    UAR_scores = []
    f1_scores = []
    max_prompt = ''
    best_UAR = 0
    f1_of_best_UAR = 0
    for prompts_id in range(5):
        data = load_from_disk(os.path.join(exp_path,'pred_dataset',f'data_prompt_{prompts_id}'))
        if lang:
            data = data.filter(lambda x: x['lang']==lang)
        y_ref = data['dx']
        y_pred = data['dx_pred']
        uar = recall_score(y_true=y_ref, y_pred=y_pred, average='macro')
        f1 = f1_score(y_true=y_ref, y_pred=y_pred, average='macro')
        UAR_scores.append(uar)
        f1_scores.append(f1)
        if uar > best_UAR:
            max_prompt = prompts_id
            best_UAR = uar
            f1_of_best_UAR = f1
    return(np.average(UAR_scores), np.std(UAR_scores), np.average(f1_scores), np.std(f1_scores), max_prompt, best_UAR, f1_of_best_UAR)

In [ ]:

for exp in os.listdir('results'):
    exp_path = os.path.join('results',exp)
    output = get_per_one_word(exp_path)
    for item in output:
        print(exp, item[0], item[1])

In [299]:
Exp_path = 'results/exp10/'
print(compute_metrics(Exp_path, lang='English'))
print(compute_metrics(Exp_path, lang='Mandarin'))
print(compute_metrics(Exp_path))

(np.float64(0.5183507549361208), np.float64(0.013920818770241691), np.float64(0.4777249684743978), np.float64(0.04442557437901736), 3, 0.537940379403794, 0.5206043549103836)
(np.float64(0.563131313131313), np.float64(0.05462043566276347), np.float64(0.47401821734647775), np.float64(0.11495056995365743), 4, 0.6363636363636364, 0.6362396806901851)
(np.float64(0.532989352989353), np.float64(0.031071660604493307), np.float64(0.4768675280124549), np.float64(0.07639073987488168), 4, 0.5824733824733825, 0.5733585445215776)


In [292]:
np.count_nonzero(np.asarray(['MCI','NC']) == 'MCI' )

1

In [281]:
y_pred_all_prompts = []
for prompts_id in range(5):
    data = load_from_disk(os.path.join(Exp_path,'pred_dataset',f'data_prompt_{prompts_id}'))
    y_ref = data['dx']
    y_pred = data['dx_pred']
    y_pred_all_prompts.append(y_pred)

In [272]:
np.asarray(y_pred_all_prompts)[:,4]

array(['MCI', 'MCI', 'MCI', 'MCI', 'MCI'], dtype='<U3')

In [ ]:
data['lang']

In [276]:
data

Dataset({
    features: ['age', 'sex', 'mmse', 'dx', 'tkdname', 'lang', 'response', 'dx_pred'],
    num_rows: 60
})

## Zero-shot v2

In [114]:
#Load the model
from transformers import Qwen2AudioForConditionalGeneration, AutoProcessor
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct")
model = Qwen2AudioForConditionalGeneration.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct", device_map="auto")
model.tie_weights()

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:09<00:00,  1.97s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


In [115]:
def prompt_with_audio(audio, prompt, model, processor):
    conversation = [
        {'role': 'system', 'content': 'You are a helpful assistant.'}, 
        {"role": "user", "content": [
            {"type": "audio", "audio": "audio.wav"},
            {"type": "text", "text": prompt},
        ]},
    ]
    text = processor.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False)
    audios = [audio]
    inputs = processor(text=text, audios=audios, return_tensors="pt", padding=True, sampling_rate=16_000).to('cuda')
    #inputs.input_ids = inputs.input_ids.to("cuda")
    generate_ids = model.generate(**inputs, max_length=1024)
    generate_ids = generate_ids[:, inputs.input_ids.size(1):] # to get only the generated text
    response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return response

In [116]:
# Load Dataset
from datasets import load_from_disk
taukadial_data = load_from_disk("/media/unsw/17D1-7AB7/mostafa/datasets/TAUKADIAL_dataset_lang")

In [117]:
def detect_CI_batch(batch, prompt, model, processor):
    audio = batch['audio']["array"]
    response = prompt_with_audio(audio, prompt, model, processor)
    batch['response'] = response
    if "MCI" in response:
        batch['dx_pred'] = "MCI"
    elif "NC" in response:
        batch['dx_pred'] = "NC"
    else:
        batch['dx_pred'] = "Unknown"
    return batch

In [118]:
def get_class_results(data, ref_col, pred_col):
    return classification_report(data[ref_col], data[pred_col], output_dict=True)  

In [12]:
PROMPT_CI = "Based on the input audio, indicate only the diagnosis. \
Choose from: Normal Cognitive Decline or Mild Cognitive Impairment. \
Give the answer as a single word. Use NC for Normal Cognitive Decline and MCI for Mild Cognitive Impairment"

In [119]:
def load_prompts(prompts_file):
    with open(prompts_file) as f:
        prompts = f.read().splitlines()
    return prompts

In [127]:
prompts = load_prompts("prompts/prompts_set2_zero_shot_CoT.txt")

In [128]:
prompts = prompts[:1]

In [129]:
prompts

['First, transcribe the speech from the input audio. Then, analyze both the audio and the transcription to determine the diagnosis. Choose from: NC for Normal Cognitive Decline or MCI for Mild Cognitive Impairment. Provide your response as a single word: NC or MCI.']

In [132]:
from datasets import Dataset
data_part = Dataset.from_dict(taukadial_data['test'][:10])

In [136]:
data_part.save_to_disk('/media/unsw/17D1-7AB7/mostafa/datasets/TAUKADIAL_dataset_10samples')

Saving the dataset (1/1 shards): 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 14.57 examples/s]


In [133]:
data_part = data_part.map(detect_CI_batch, fn_kwargs={'prompt': prompts[0],'model':model, 'processor':processor});

Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [43:25<00:00, 260.59s/ examples]


In [135]:
data_part['response']

['MCI',
 'MCI',
 "The speech content is '这个小孩子就玩电吧，把这个大人把把叫。那个狗汪汪叫，把这个猫咪听他讲讲，舌头伸出来。那这个是什么意思？是那个那个花瓶水倒下来，大概这样吧。'",
 "The speech in the audio is: 'Okay, there was a family who were going on a trip mother, father, and three kids and they're driving along and the kids one little boy is hanging out the window with his dog and the little girl is chewing bubble gum working on a big making a big bubble and the other little boy is waving at people and then they keep driving until it gets to be dark and the mother is asleep in the front seat and the one of the girls is asleep in the back.' Based on this description, it seems that the family is just going on a routine trip, without any signs of cognitive decline or impairment. Therefore, the diagnosis would be NC (Normal Cognitive Decline).",
 "The speech in the audio is transcribed as follows: 'the little girl is riding her tricycle and her dad is walking along with her and the cat and the cat comes out and runs up the tree because the dog is ch

In [17]:
from sklearn.metrics import classification_report
d = classification_report(data_part['dx'], data_part['dx_detect'], output_dict=True)

/home/unsw/anaconda3/envs/qa/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/unsw/anaconda3/envs/qa/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/unsw/anaconda3/envs/qa/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

In [31]:
from sklearn.metrics import classification_report
import pandas as pd
from copy import deepcopy
results = []
pred_dataset = []
for prompt in prompts:
    print(prompt)
    data_pred = taukadial_data['test'].map(detect_CI_batch, fn_kwargs={'prompt': prompt,'model':model, 'processor':processor});
    pred_dataset.append(deepcopy(data_pred))
    results_d = get_class_results(data_pred, 'dx','dx_pred')
    results.append({"prompt":prompt, 
                    "MCI_recall":results_d["MCI"]["recall"],
                   "MCI_precision":results_d["MCI"]["precision"],
                   "MCI_f1-score":results_d["MCI"]["f1-score"],
                   "MCI_support":results_d["MCI"]["support"],
                   "NC_recall":results_d["NC"]["recall"],
                   "NC_precision":results_d["NC"]["precision"],
                   "NC_f1-score":results_d["NC"]["f1-score"],
                   "NC_support":results_d["NC"]["support"],
                   "f1_score_macro":results_d['macro avg']['f1-score'],
                   "f1_score_weighted":results_d['weighted avg']['f1-score']})
df = pd.DataFrame.from_records(results)

Analyze the input audio and determine the diagnosis. Select from the following options: NC for Normal Cognitive Decline or MCI for Mild Cognitive Impairment. Provide your response as a single word: NC or MCI.


Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 120/120 [23:03<00:00, 11.53s/ examples]
/home/unsw/anaconda3/envs/qa/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/unsw/anaconda3/envs/qa/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/unsw/anaconda3/envs/qa/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being

Listen to the provided audio and classify the cognitive condition.            Choose either NC for Normal Cognitive Decline or MCI for Mild Cognitive Impairment.            Respond with only one word: NC or MCI.


Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 120/120 [23:10<00:00, 11.58s/ examples]


Based on the speech patterns in the input audio, identify the cognitive status.            Use NC for Normal Cognitive Decline or MCI for Mild Cognitive Impairment.            Your response should be a single word: NC or MCI.


...

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 120/120 [21:51<00:00, 10.93s/ examples]


Assess the cognitive condition based on the input audio.            Indicate the diagnosis using one of these labels: NC for Normal Cognitive Decline or MCI for Mild Cognitive Impairment.            Output only NC or MCI as your response.


Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 120/120 [30:05<00:00, 15.05s/ examples]


Based on the input audio, indicate only the diagnosis.            Choose from: Normal Cognitive Decline or Mild Cognitive Impairment.            Give the answer as a single word. Use NC for Normal Cognitive Decline and MCI for Mild Cognitive Impairment.


Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 120/120 [32:46<00:00, 16.39s/ examples]
/home/unsw/anaconda3/envs/qa/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/unsw/anaconda3/envs/qa/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/unsw/anaconda3/envs/qa/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being

In [41]:
df.to_csv('tmp.csv')

In [35]:
results_d

{'MCI': {'precision': 0.525,
  'recall': 1.0,
  'f1-score': 0.6885245901639344,
  'support': 63.0},
 'NC': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 57.0},
 'accuracy': 0.525,
 'macro avg': {'precision': 0.2625,
  'recall': 0.5,
  'f1-score': 0.3442622950819672,
  'support': 120.0},
 'weighted avg': {'precision': 0.275625,
  'recall': 0.525,
  'f1-score': 0.3614754098360656,
  'support': 120.0}}

In [36]:
results_d['weighted avg']['f1-score']

0.3442622950819672

In [39]:
df.to_csv('tmp.csv', index=False)

In [15]:
from copy import deepcopy
x = deepcopy(data_part)

In [7]:
data_part.remove_columns(['audio']).save_to_disk('data_part_x')

Saving the dataset (1/1 shards): 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 836.52 examples/s]


In [6]:
data_part_x

Dataset({
    features: ['age', 'sex', 'mmse', 'dx', 'tkdname', 'lang'],
    num_rows: 10
})

# Zero-shot with customized info

In [1]:
#Load the model
from transformers import Qwen2AudioForConditionalGeneration, AutoProcessor
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct")
model = Qwen2AudioForConditionalGeneration.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct", device_map="auto")
model.tie_weights()

/home/unsw/anaconda3/envs/qa/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:14<00:00,  2.92s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


In [2]:
def prompt_with_audio(audio, prompt, model, processor):
    conversation = [
        {'role': 'system', 'content': 'You are a helpful assistant.'}, 
        {"role": "user", "content": [
            {"type": "audio", "audio": "audio.wav"},
            {"type": "text", "text": prompt},
        ]},
    ]
    text = processor.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False)
    audios = [audio]
    inputs = processor(text=text, audios=audios, return_tensors="pt", padding=True, sampling_rate=16_000).to('cuda')
    #inputs.input_ids = inputs.input_ids.to("cuda")
    generate_ids = model.generate(**inputs, max_length=1024)
    generate_ids = generate_ids[:, inputs.input_ids.size(1):] # to get only the generated text
    response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return response

In [17]:
# Load Dataset
from datasets import load_from_disk
taukadial_data = load_from_disk("/media/unsw/17D1-7AB7/mostafa/datasets/TAUKADIAL_dataset_10samples/")

In [18]:
taukadial_data

Dataset({
    features: ['audio', 'age', 'sex', 'mmse', 'dx', 'tkdname', 'lang'],
    num_rows: 10
})

In [25]:
gender_map = {'M':'male','F':'female'}
def detect_CI_with_filling_batch(batch, prompt, model, processor):
    audio = batch['audio']["array"]
    if '[AGE]' in prompt:
        prompt = prompt.replace('[AGE]',str(batch['age']))
    if '[LANGUAGE]' in prompt:
        prompt = prompt.replace('[LANGUAGE]',batch['lang'])
    if '[GENDER]' in prompt:
        prompt = prompt.replace('[GENDER]', gender_map[batch['sex']])
    batch['prompt'] = prompt
    response = prompt_with_audio(audio, prompt, model, processor)
    batch['response'] = response
    if "MCI" in response:
        batch['dx_pred'] = "MCI"
    elif "NC" in response:
        batch['dx_pred'] = "NC"
    else:
        batch['dx_pred'] = "Unknown"
    return batch

In [20]:
def get_class_results(data, ref_col, pred_col):
    return classification_report(data[ref_col], data[pred_col], output_dict=True)  

In [12]:
PROMPT_CI = "Based on the input audio, indicate only the diagnosis. \
Choose from: Normal Cognitive Decline or Mild Cognitive Impairment. \
Give the answer as a single word. Use NC for Normal Cognitive Decline and MCI for Mild Cognitive Impairment"

In [21]:
def load_prompts(prompts_file):
    with open(prompts_file) as f:
        prompts = f.read().splitlines()
    return prompts

In [22]:
prompts = load_prompts("tmp_prompt.txt")

In [23]:
prompts

['Analyze the input audio, in which a [AGE]-year-old [GENDER] describes a picture in [LANGUAGE] as prompted by a clinician. Determine the diagnosis based on speech patterns. Select from: NC for Normal Cognitive Decline or MCI for Mild Cognitive Impairment. Provide your response as a single word: NC or MCI.',
 'Listen to the provided audio, in which a [AGE]-year-old [GENDER] describes a picture in [LANGUAGE] as prompted by a clinician. Classify the cognitive condition. Choose either NC for Normal Cognitive Decline or MCI for Mild Cognitive Impairment. Respond with only one word: NC or MCI.',
 'Based on the speech patterns in the input audio, in which a [AGE]-year-old [GENDER] describes a picture in [LANGUAGE], identify the cognitive status. Use NC for Normal Cognitive Decline or MCI for Mild Cognitive Impairment. Your response should be a single word: NC or MCI.',
 'From the given speech sample, in which a [AGE]-year-old [GENDER] describes a picture in [LANGUAGE] as prompted by a clinic

In [26]:
data_part = taukadial_data.map(detect_CI_with_filling_batch, fn_kwargs={'prompt': prompts[0],'model':model, 'processor':processor});

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.                         | 0/10 [00:00<?, ? examples/s]
Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:49<00:00, 10.98s/ examples]


In [30]:
data_part['dx']

['NC', 'NC', 'NC', 'MCI', 'MCI', 'MCI', 'NC', 'NC', 'NC', 'NC']

# Draft

In [46]:
pwd

'/media/unsw/17D1-7AB7/mostafa/audioLLM/Qwen'

In [73]:
import numpy as np
max_length = processor.feature_extractor.n_samples
def process_audio( audio_url: str) -> np.ndarray:
    try:
        audio, sr = librosa.load(BytesIO(urlopen(audio_url).read()), sr=processor.feature_extractor.sampling_rate)
        
        if len(audio) > max_length:
            if debug:
                print(f"Trimming audio to {max_length} samples")
            audio = audio[:max_length]
        return audio.astype(np.float32)
        
    except Exception as e:
        raise RuntimeError(f"Error processing audio: {e}")

In [74]:
eval_dataset

Dataset({
    features: ['id', 'name', 'url', 'license', '__index_level_0__'],
    num_rows: 9
})

In [75]:
valid_examples = []
audios = []
combined_texts = []
task_prompt = "Identify the bird species in the audio clip"
for example in eval_dataset:
    audio = process_audio(example["url"])
    audios.append(audio)

    conversation = [
        {"role": "user", "content":[
            {"type": "audio", "audio_url": example["url"]},
            {"type": "text", "text": task_prompt}
        ]},
        {"role": "assistant", "content": example["name"]}
    ]
    combined_text = processor.apply_chat_template(
        conversation, add_generation_prompt=False, tokenize=False  #Here add_generation_prompt=False as it's training and the answer is in the conversation, and tokenize=false as we will do tokenize later
    )

    combined_texts.append(combined_text)
    valid_examples.append(example)

In [79]:
len(audios)

9

In [81]:
inputs = processor(
                text=combined_texts,
                audios=audios,
                return_tensors="pt",
                padding=True
            )

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [84]:
processor.tokenizer.convert_ids_to_tokens([151646])

['<|AUDIO|>']

In [85]:
assistant_start_tokens = processor.tokenizer.encode("<|im_start|>assistant")

In [86]:
assistant_start_tokens

[151644, 77091]

In [118]:
inputs

{'input_ids': tensor([[151643, 151644,   8948,  ...,  40652, 151645,    198],
        [151643, 151644,   8948,  ...,   7183, 151645,    198],
        [151643, 151643, 151643,  ...,  77462, 151645,    198],
        ...,
        [151643, 151643, 151643,  ...,   2468, 151645,    198],
        [151644,   8948,    198,  ...,  11346, 151645,    198],
        [151644,   8948,    198,  ...,    838, 151645,    198]]), 'attention_mask': tensor([[0, 1, 1,  ..., 1, 1, 1],
        [0, 1, 1,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        ...,
        [0, 0, 0,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'input_features': tensor([[[-1.3869, -1.3869, -1.3869,  ..., -1.3869, -1.3869, -1.3869],
         [-1.3869, -1.3869, -1.3869,  ..., -1.3869, -1.3869, -1.3869],
         [-1.3869, -1.3869, -1.3869,  ..., -1.3869, -1.3869, -1.3861],
         ...,
         [-1.3869, -1.3869, -1.3869,  ..., -1.3100, -1.3505, -1.2018],
         [-1.3869, -1.3869, -1.386

In [ ]:
#load audio
TASK_PROMPT = "What type of bird do you hear? output only the type of the bird"
audio_url = "https://xeno-canto.org/841748/download"
audio_data, sample_rate = librosa.load(BytesIO(urlopen(audio_url).read()), sr=None)
#play audio
display(Audio(audio_data, rate=sample_rate))
conversation = [
    {"role": "user", "content": [
        {"type": "audio", "audio_url": audio_url},
        {"type": "text", "text": TASK_PROMPT},
    ]}
]
# process conversation
text = processor.apply_chat_template(conversation, add_generation_prompt= True, tokenize=False)
if debug:
    print(f"Templated text:\n{text}")
    
audios = []

for message in conversation:
    if isinstance(message["content"], list):
        for item in message["content"]:
            if isinstance(item, dict) and item["type"] == "audio" and "audio_url" in item:
                try:
                    sr = int(processor.feature_extractor.sampling_rate)
                    aduio_data = librosa.load(BytesIO(urlopen(item["audio_url"]).read()), sr=sr)[0]
                    audios.append(aduio_data)
                except Exception as e:
                    print(f"failed to process audio from {item['audio_url']}, {e}")
inputs = processor(
    text=text,
    audios=audios,
    return_tensors="pt",
    padding=True,
).to("cuda")

generate_ids = model.generate(**inputs, max_length=512)

print(f"Input shape: {inputs.input_ids.size(1)}, Generated shape: {generate_ids.shape}")
generate_ids = generate_ids[:, inputs.input_ids.size(1):]

response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

In [17]:
response

'The bird is a sparrow.'

In [301]:
!which python

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/unsw/anaconda3/envs/qa/bin/python


In [1]:
from datasets import load_from_disk

/home/unsw/anaconda3/envs/qa/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
data = load_from_disk('results/tmp/pred_dataset/data_prompt_4/')

In [17]:
data['dx_pred']

['NC', 'MCI', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC']

In [18]:
data['dx']

['NC', 'NC', 'NC', 'MCI', 'MCI', 'MCI', 'NC', 'NC', 'NC', 'MCI']

In [19]:
data['prompt']

['Assess the cognitive condition based on the input audio, in which a 70-year-old female describes the Cookie Theft picture in Mandarin as part of a clinician-guided task. Analyze both linguistic and acoustic features, including pauses, hesitations, and richness of vocabulary. The dataset includes recordings in both Chinese and English. Indicate the diagnosis using one of these labels: NC for Normal Cognitive Decline or MCI for Mild Cognitive Impairment. Output only NC or MCI as your response.',
 'Assess the cognitive condition based on the input audio, in which a 70-year-old female describes the Cookie Theft picture in Mandarin as part of a clinician-guided task. Analyze both linguistic and acoustic features, including pauses, hesitations, and richness of vocabulary. The dataset includes recordings in both Chinese and English. Indicate the diagnosis using one of these labels: NC for Normal Cognitive Decline or MCI for Mild Cognitive Impairment. Output only NC or MCI as your response.'